# 1) 데이터 준비와 전처리

In [1]:
import pandas as pd
import os
rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python', encoding = "ISO-8859-1")
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [2]:
# 3점 이상만 남깁니다.
ratings = ratings[ratings['rating']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [3]:
# rating 컬럼의 이름을 count로 바꿉니다.
ratings.rename(columns={'rating':'count'},inplace=True)

In [4]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding='ISO-8859-1')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [5]:
ratings=pd.merge(ratings, movies, on="movie_id", how="left")
ratings

,user_id,movie_id,count,timestamp,title,genre
0,1,1193,5,978300760,One Flew Over the Cuckoo's Nest (1975),Drama
1,1,661,3,978302109,James and the Giant Peach (1996),Animation|Children's|Musical
2,1,914,3,978301968,My Fair Lady (1964),Musical|Romance
3,1,3408,4,978300275,Erin Brockovich (2000),Drama
4,1,2355,5,978824291,"Bug's Life, A (1998)",Animation|Children's|Comedy
...,...,...,...,...,...,...
836473,6040,1090,3,956715518,Platoon (1986),Drama|War
836474,6040,1094,5,956704887,"Crying Game, The (1992)",Drama|Romance|War
836475,6040,562,5,956704746,Welcome to the Dollhouse (1995),Comedy|Drama
836476,6040,1096,4,956715648,Sophie's Choice (1982),Drama


In [6]:
# 필요없는 정보 삭제
using_cols=['user_id','title','count']
ratings=ratings[using_cols]
ratings.head(10)

,user_id,title,count
0,1,One Flew Over the Cuckoo's Nest (1975),5
1,1,James and the Giant Peach (1996),3
2,1,My Fair Lady (1964),3
3,1,Erin Brockovich (2000),4
4,1,"Bug's Life, A (1998)",5
5,1,"Princess Bride, The (1987)",3
6,1,Ben-Hur (1959),5
7,1,"Christmas Story, A (1983)",5
8,1,Snow White and the Seven Dwarfs (1937),4
9,1,"Wizard of Oz, The (1939)",4


In [7]:
# title 컬럼의 이름을 movie로 바꿉니다.
ratings.rename(columns={'title':'movie'},inplace=True)

/opt/conda/lib/python3.7/site-packages/pandas/core/frame.py:4446: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


# 2) 분석해 봅시다.

- ratings에 있는 유니크한 영화 개수

In [8]:
ratings['movie'].nunique()

3628

ratings에 있는 유니크한 영화의 개수는 3628개이다.

- ratings에 있는 유니크한 사용자 수

In [9]:
ratings['user_id'].nunique()

6039

ratings에 있는 유니크한 사용사의 수는 6040명이다.

- 가장 인기 있는 영화 30개(인기순)

In [10]:
movie_count=ratings.groupby('movie')['user_id'].count()
movie_count.sort_values(ascending=False).head(30)

movie
American Beauty (1999)                                   3211
Star Wars: Episode IV - A New Hope (1977)                2910
Star Wars: Episode V - The Empire Strikes Back (1980)    2885
Star Wars: Episode VI - Return of the Jedi (1983)        2716
Saving Private Ryan (1998)                               2561
Terminator 2: Judgment Day (1991)                        2509
Silence of the Lambs, The (1991)                         2498
Raiders of the Lost Ark (1981)                           2473
Back to the Future (1985)                                2460
Matrix, The (1999)                                       2434
Jurassic Park (1993)                                     2413
Sixth Sense, The (1999)                                  2385
Fargo (1996)                                             2371
Braveheart (1995)                                        2314
Men in Black (1997)                                      2297
Schindler's List (1993)                                  2257
Pr

# 3) 내가 선호하는 영화를 5가지 골라서 rating에 추가해 줍시다.

In [11]:
my_favorite = ['Toy Story (1995)', 'E.T. the Extra-Terrestrial (1982)', 'Star Wars: Episode IV - A New Hope (1977)', 'Matrix, The (1999)', 'Silence of the Lambs, The (1991)']

my_playlist = pd.DataFrame({'user_id': ['yejin']*5, 'movie': my_favorite, 'count':[5]*5})

if not ratings.isin({'user_id':['yejin']})['user_id'].any():  # user_id에 'yejin'이라는 데이터가 없다면
    ratings = ratings.append(my_playlist)                           # 위에 임의로 만든 my_favorite 데이터를 추가해 줍니다. 

ratings.tail(10)

,user_id,movie,count
836473,6040,Platoon (1986),3
836474,6040,"Crying Game, The (1992)",5
836475,6040,Welcome to the Dollhouse (1995),5
836476,6040,Sophie's Choice (1982),4
836477,6040,E.T. the Extra-Terrestrial (1982),4
0,yejin,Toy Story (1995),5
1,yejin,E.T. the Extra-Terrestrial (1982),5
2,yejin,Star Wars: Episode IV - A New Hope (1977),5
3,yejin,"Matrix, The (1999)",5
4,yejin,"Silence of the Lambs, The (1991)",5


# 4) CSR matrix를 직접 만들어 봅시다.

In [12]:
#고유한 유저, 아티스트를 찾아내는 코드
user_unique=ratings['user_id'].unique()
movie_unique=ratings['movie'].unique()

#유저, 아티스트 indexing 하는 코드 idx는 index의 약자
user_to_idx={v:k for k,v in enumerate(user_unique)}
movie_to_idx={v:k for k,v in enumerate(movie_unique)}

In [13]:
# 인덱싱이 잘 되었는지 확인해 봅니다. 
print(user_to_idx['yejin']) 
print(movie_to_idx['Platoon (1986)'])

6039
1030


In [14]:
temp_user_data = ratings['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(ratings):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    ratings['user_id'] = temp_user_data   # data['user_id']을 인덱싱된 Series로 교체해 줍니다. 
else:
    print('user_id column indexing Fail!!')

# movie_to_idx을 통해 movie_id 컬럼도 동일한 방식으로 인덱싱해 줍니다. 
temp_movie_data = ratings['movie'].map(movie_to_idx.get).dropna()
if len(temp_movie_data) == len(ratings):
    print('movie column indexing OK!!')
    ratings['movie'] = temp_movie_data
else:
    print('movie column indexing Fail!!')

ratings

user_id column indexing OK!!
movie column indexing OK!!


,user_id,movie,count
0,0,0,5
1,0,1,3
2,0,2,3
3,0,3,4
4,0,4,5
...,...,...,...
0,6039,40,5
1,6039,26,5
2,6039,44,5
3,6039,124,5


In [27]:
from scipy.sparse import csr_matrix

num_user = ratings['user_id'].nunique()
num_movie = ratings['movie'].nunique()

csr_data = csr_matrix((ratings['count'], (ratings['user_id'], ratings['movie'])), shape=(num_user, num_movie) )
csr_data

<6040x3628 sparse matrix of type '<class 'numpy.longlong'>'
	with 836483 stored elements in Compressed Sparse Row format>

# 5) als_model = AlternatingLeastSquares 모델을 직접 구성하여 훈련시켜 봅시다.

In [28]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

# implicit 라이브러리에서 권장하고 있는 부분입니다. 학습 내용과는 무관합니다.
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

In [29]:
# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=10, dtype=np.float32)

In [30]:
# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)
csr_data_transpose=csr_data.T
csr_data_transpose

<3628x6040 sparse matrix of type '<class 'numpy.longlong'>'
	with 836483 stored elements in Compressed Sparse Column format>

In [31]:
als_model.fit(csr_data_transpose)

  0%|          | 0/10 [00:00<?, ?it/s]

# 6) 내가 선호하는 5가지 영화 중 하나와 그 외의 영화 하나를 골라 훈련된 모델이 예측한 나의 선호도를 파악해 보세요.

In [38]:
# 모델이 예측한 나의 선호도
# 1. 내가 선호하는 영화인 Toy Story (1995)에 대한 선호도
yejin, Toy_Story = user_to_idx['yejin'], movie_to_idx['Toy Story (1995)']
yejin_vector, Toy_Story_vector = als_model.user_factors[yejin], als_model.item_factors[Toy_Story]

In [41]:
np.dot(yejin_vector, Toy_Story_vector)

0.49298796

In [43]:
#2. 내가 선호하지 않는 영화인 Ghostbusters (1984)에 대한 선호도
Ghostbusters = movie_to_idx['Ghostbusters (1984)']
Ghostbusters_vector = als_model.item_factors[Ghostbusters]

In [44]:
np.dot(yejin_vector, Ghostbusters_vector)

0.13597515

확실히 선호하는 영화에 대한 값은 0.49298796로 선호하지 않은 영화에 대한 값인 0.13597515 보다 훨씬 컸다.

# 7) 내가 좋아하는 영화와 비슷한 영화를 추천받아 봅시다.

In [51]:
#movie_to_idx 를 뒤집어, index로부터 movie 이름을 얻는 dict를 생성합니다. 
idx_to_movie = {v:k for k,v in movie_to_idx.items()}

def get_similar_movie(movie_title: str):
    movie_id = movie_to_idx[movie_title]
    similar_movie = als_model.similar_items(movie_id)
    similar_movie = [idx_to_movie[i[0]] for i in similar_movie]
    return similar_movie

get_similar_movie('Toy Story (1995)')

['Toy Story (1995)',
 'Toy Story 2 (1999)',
 'Babe (1995)',
 "Bug's Life, A (1998)",
 'Aladdin (1992)',
 'Groundhog Day (1993)',
 'Lion King, The (1994)',
 'Beauty and the Beast (1991)',
 'Pleasantville (1998)',
 "There's Something About Mary (1998)"]

# 8) 내가 가장 좋아할 만한 영화들을 추천받아 봅시다.

In [55]:
user = user_to_idx['yejin']
# recommend에서는 user*item CSR Matrix를 받습니다.
movie_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items=True)
[idx_to_movie[i[0]] for i in movie_recommended]

['Star Wars: Episode V - The Empire Strikes Back (1980)',
 'Star Wars: Episode VI - Return of the Jedi (1983)',
 'Terminator 2: Judgment Day (1991)',
 'Back to the Future (1985)',
 'Shawshank Redemption, The (1994)',
 'Babe (1995)',
 'Saving Private Ryan (1998)',
 'Star Wars: Episode I - The Phantom Menace (1999)',
 'Sixth Sense, The (1999)',
 "Schindler's List (1993)",
 'Forrest Gump (1994)',
 'Toy Story 2 (1999)',
 'Fargo (1996)',
 'Raiders of the Lost Ark (1981)',
 'Terminator, The (1984)',
 'American Beauty (1999)',
 'Fugitive, The (1993)',
 'Close Encounters of the Third Kind (1977)',
 '2001: A Space Odyssey (1968)',
 'Groundhog Day (1993)']

# 회고
- 처음에는 ratings과 movies를 합칠 생각을 못했다. csr_matrix를 할 때 movie의 인덱스보다 movie 제목을 바로 넣으면 좋을 것 같다는 생각이 들었고, 시도해봤다.
- len() of unsized object 에러가 나와서 한참 헤메다가 ratings.count를 ratings[count]로 수정하니깐 에러가 해결되었다.아직 이유는 잘 모르겠다.

- Matrix Factorization(MF)와 CSR(Compressed Sparse Row)Matrix가 잘 이해가 가지 않아서 몇시간 동안 찾아봤지만 아직도 이해가 완전히 가지 않는다. 나중에 다시 공부해야될것 같다.
- 추천시스템을 이용만 해보다가 직접 간단하게 구현을 해보니깐 신기했다.